### Top 3 Salaries per Department
### You have a dataset: employee_id, name, salary, department. Write a query to get the top 3 highest-paid employees in each department.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, dense_rank

# Step 2: Sample employee data
data = [11212222222222222222222222222222222222222222222222222222222222222222222222aq                                                                qww
    (1, "Alice", 50000, "HR"),
    (2, "Bob", 60000, "HR"),
    (3, "Charlie", 55000, "HR"),
    (4, "David", 70000, "IT"),
    (5, "Eve", 85000, "IT"),
    (6, "Frank", 90000, "IT"),
    (7, "Grace", 75000, "IT"),
    (8, "Hank", 45000, "Sales"),
    (9, "Ivy", 47000, "Sales"),
    (10, "Jack", 46000, "Sales"),
    (11, "Kara", 49000, "Sales")
]

columns = ["employee_id", "name", "salary", "department"]
df = spark.createDataFrame(data, columns)

# Step 3: Define window partitioned by department and ordered by descending salary
windowSpec = Window.partitionBy("department").orderBy(col("salary").desc())

# Step 4: Apply dense_rank to assign rank within each department
ranked_df = df.withColumn("rank", dense_rank().over(windowSpec))

# Step 5: Filter top 3 salaries per department
top3_df = ranked_df.filter(col("rank") <= 3)

# Step 6: Show result
top3_df.select("employee_id", "name", "salary", "department", "rank").show()


+-----------+-------+------+----------+----+
|employee_id|   name|salary|department|rank|
+-----------+-------+------+----------+----+
|          2|    Bob| 60000|        HR|   1|
|          3|Charlie| 55000|        HR|   2|
|          1|  Alice| 50000|        HR|   3|
|          6|  Frank| 90000|        IT|   1|
|          5|    Eve| 85000|        IT|   2|
|          7|  Grace| 75000|        IT|   3|
|         11|   Kara| 49000|     Sales|   1|
|          9|    Ivy| 47000|     Sales|   2|
|         10|   Jack| 46000|     Sales|   3|
+-----------+-------+------+----------+----+



### 2. Handling Salary Ties with rank()
### In the same employee dataset, you want to rank employees by salary within each department, and account for ties (equal salaries should get the same rank, and next rank should skip).

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank

# Step 2: Sample employee data with salary ties
data = [
    (1, "Alice", 50000, "HR"),
    (2, "Bob", 60000, "HR"),
    (3, "Charlie", 60000, "HR"),
    (4, "David", 70000, "IT"),
    (5, "Eve", 85000, "IT"),
    (6, "Frank", 90000, "IT"),
    (7, "Grace", 85000, "IT"),
    (8, "Hank", 45000, "Sales"),
    (9, "Ivy", 47000, "Sales"),
    (10, "Jack", 46000, "Sales"),
    (11, "Kara", 47000, "Sales")
]

columns = ["employee_id", "name", "salary", "department"]
df = spark.createDataFrame(data, columns)

# Step 3: Define window partitioned by department and ordered by descending salary
windowSpec = Window.partitionBy("department").orderBy(col("salary").desc())

# Step 4: Apply rank to handle salary ties (same salary gets the same rank, next rank skips)
ranked_df = df.withColumn("rank", rank().over(windowSpec))

# Step 5: Show the result
ranked_df.select("employee_id", "name", "salary", "department", "rank").show()


+-----------+-------+------+----------+----+
|employee_id|   name|salary|department|rank|
+-----------+-------+------+----------+----+
|          2|    Bob| 60000|        HR|   1|
|          3|Charlie| 60000|        HR|   1|
|          1|  Alice| 50000|        HR|   3|
|          6|  Frank| 90000|        IT|   1|
|          5|    Eve| 85000|        IT|   2|
|          7|  Grace| 85000|        IT|   2|
|          4|  David| 70000|        IT|   4|
|          9|    Ivy| 47000|     Sales|   1|
|         11|   Kara| 47000|     Sales|   1|
|         10|   Jack| 46000|     Sales|   3|
|          8|   Hank| 45000|     Sales|   4|
+-----------+-------+------+----------+----+



### 3.Avoid Gaps in Rank using dense_rank()
### If your business needs consecutive ranking even for tied salaries (no rank gaps), how would you adjust the logic?

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, dense_rank

# Step 2: Sample employee data with salary ties
data = [
    (1, "Alice", 50000, "HR"),
    (2, "Bob", 60000, "HR"),
    (3, "Charlie", 60000, "HR"),
    (4, "David", 70000, "IT"),
    (5, "Eve", 85000, "IT"),
    (6, "Frank", 90000, "IT"),
    (7, "Grace", 85000, "IT"),
    (8, "Hank", 45000, "Sales"),
    (9, "Ivy", 47000, "Sales"),
    (10, "Jack", 46000, "Sales"),
    (11, "Kara", 47000, "Sales")
]

columns = ["employee_id", "name", "salary", "department"]
df = spark.createDataFrame(data, columns)

# Step 3: Define window partitioned by department and ordered by descending salary
windowSpec = Window.partitionBy("department").orderBy(col("salary").desc())

# Step 4: Apply dense_rank to avoid gaps in ranks (consecutive ranking even with ties)
dense_ranked_df = df.withColumn("rank", dense_rank().over(windowSpec))

# Step 5: Show the result
dense_ranked_df.select("employee_id", "name", "salary", "department", "rank").show()

+-----------+-------+------+----------+----+
|employee_id|   name|salary|department|rank|
+-----------+-------+------+----------+----+
|          2|    Bob| 60000|        HR|   1|
|          3|Charlie| 60000|        HR|   1|
|          1|  Alice| 50000|        HR|   2|
|          6|  Frank| 90000|        IT|   1|
|          5|    Eve| 85000|        IT|   2|
|          7|  Grace| 85000|        IT|   2|
|          4|  David| 70000|        IT|   3|
|          9|    Ivy| 47000|     Sales|   1|
|         11|   Kara| 47000|     Sales|   1|
|         10|   Jack| 46000|     Sales|   2|
|          8|   Hank| 45000|     Sales|   3|
+-----------+-------+------+----------+----+



### 4. Find Highest Salary Employee per Department
### Find only the highest-paid employee(s) in each department. Use a window function to do it (not groupBy).

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank

# Step 2: Sample employee data
data = [
    (1, "Alice", 50000, "HR"),
    (2, "Bob", 60000, "HR"),
    (3, "Charlie", 70000, "HR"),
    (4, "David", 90000, "IT"),
    (5, "Eve", 85000, "IT"),
    (6, "Frank", 90000, "IT"),
    (7, "Grace", 95000, "IT"),
    (8, "Hank", 45000, "Sales"),
    (9, "Ivy", 47000, "Sales"),
    (10, "Jack", 46000, "Sales"),
    (11, "Kara", 50000, "Sales")
]

columns = ["employee_id", "name", "salary", "department"]
df = spark.createDataFrame(data, columns)

# Step 3: Define window specification partitioned by department and ordered by salary descending
windowSpec = Window.partitionBy("department").orderBy(col("salary").desc())

# Step 4: Apply rank function to rank employees by salary within each department
ranked_df = df.withColumn("rank", rank().over(windowSpec))

# Step 5: Filter to get only the highest-paid employee(s) (rank = 1)
highest_paid_df = ranked_df.filter(col("rank") == 1)

# Step 6: Show the result
highest_paid_df.select("employee_id", "name", "salary", "department", "rank").show()

+-----------+-------+------+----------+----+
|employee_id|   name|salary|department|rank|
+-----------+-------+------+----------+----+
|          3|Charlie| 70000|        HR|   1|
|          7|  Grace| 95000|        IT|   1|
|         11|   Kara| 50000|     Sales|   1|
+-----------+-------+------+----------+----+



### 5. Nth Highest Salary Across Company
### You want to find the employee with the 3rd highest salary in the entire company, even if there are ties.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, dense_rank

# Step 2: Sample employee data (includes salary ties)
data = [
    (1, "Alice", 50000, "HR"),
    (2, "Bob", 60000, "HR"),
    (3, "Charlie", 70000, "HR"),
    (4, "David", 90000, "IT"),
    (5, "Eve", 85000, "IT"),
    (6, "Frank", 90000, "IT"),
    (7, "Grace", 85000, "IT"),
    (8, "Hank", 45000, "Sales"),
    (9, "Ivy", 47000, "Sales"),
    (10, "Jack", 46000, "Sales"),
    (11, "Kara", 50000, "Sales")
]

columns = ["employee_id", "name", "salary", "department"]
df = spark.createDataFrame(data, columns)

# Step 3: Define a window across the entire company (no partition), ordered by salary descending
windowSpec = Window.orderBy(col("salary").desc())

# Step 4: Apply dense_rank (avoids skipping rank numbers for ties)
ranked_df = df.withColumn("rank", dense_rank().over(windowSpec))

# Step 5: Filter where rank == 3 (3rd highest salary)
third_highest_df = ranked_df.filter(col("rank") == 3)

# Step 6: Show result
third_highest_df.select("employee_id", "name", "salary", "department", "rank").show()


+-----------+-------+------+----------+----+
|employee_id|   name|salary|department|rank|
+-----------+-------+------+----------+----+
|          3|Charlie| 70000|        HR|   3|
+-----------+-------+------+----------+----+



### 6. Detect Duplicate Records
### You suspect some employees have duplicate entries. Use row_number() on columns employee_id, name, department to detect and filter duplicates.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

# Step 2: Sample data with duplicates
data = [
    (1, "Alice", 50000, "HR"),
    (2, "Bob", 60000, "HR"),
    (3, "Charlie", 70000, "HR"),
    (4, "David", 90000, "IT"),
    (5, "Eve", 85000, "IT"),
    (6, "Frank", 90000, "IT"),
    (7, "Grace", 85000, "IT"),
    (2, "Bob", 60000, "HR"),       # duplicate
    (5, "Eve", 85000, "IT"),       # duplicate
    (8, "Hank", 45000, "Sales"),
    (9, "Ivy", 47000, "Sales"),
    (10, "Jack", 46000, "Sales"),
    (11, "Kara", 50000, "Sales")
]

columns = ["employee_id", "name", "salary", "department"]
df = spark.createDataFrame(data, columns)

# Step 3: Define window over duplicates based on employee_id, name, department
windowSpec = Window.partitionBy("employee_id", "name", "department").orderBy("salary")

# Step 4: Apply row_number() to detect duplicates
numbered_df = df.withColumn("row_number", row_number().over(windowSpec))

# Step 5: Filter duplicates (row_number > 1)
duplicates_df = numbered_df.filter(col("row_number") > 1)

# Step 6: Show duplicate rows
duplicates_df.select("employee_id", "name", "salary", "department").show()


+-----------+----+------+----------+
|employee_id|name|salary|department|
+-----------+----+------+----------+
|          2| Bob| 60000|        HR|
|          5| Eve| 85000|        IT|
+-----------+----+------+----------+

